In [126]:
import sys
import argparse
import bintrees
import msprime 
import math 

In [100]:
class Segment(object):
    """
    A class representing a single segment. Each segment has a left
    and right, denoting the loci over which it spans, a node and a
    next, giving the next in the chain.
    """
    def __init__(self, index):
        self.left = None
        self.right = None
        self.node = None
        self.prev = None
        self.next = None
        self.population = None
        self.index = index

    def __str__(self):
        s = "({0}:{1}-{2}->{3}: prev={4} next={5})".format(
            self.index, self.left, self.right, self.node, repr(self.prev),
            repr(self.next))
        print("reprt is %s" %s)
        return s
    #Q# what is this for??
    def __lt__(self, other):
        return ((self.left, self.right, self.population, self.node)
                < (other.left, other.right, other.population, self.node))

In [ ]:
s = Segment(0)

s.left=0 ;s.right = 3; s.node = 1; s.population = 0 ;s.prev = None; s.next = 0

s1 = Segment(0)
s1.left=5 ;s1.right = 10; s1.node = 1; s1.population = 0 ;s1.prev = None; s1.next = 1
# s1= Segment(2)
s.__str__()
s1.__str__()
s.__lt__(s1)

# Fenwick algorithm  or  Binary Indexed Tree

Great explanation  found at: 
https://www.geeksforgeeks.org/binary-indexed-tree-or-fenwick-tree-2/

Youtube video:
https://www.youtube.com/watch?v=v_wj_mOAlig


In [41]:
# my implementation of Binary indexed tree: 

def sum_BIT(BITree, index):
    
    '''
    Returns sum of arr[0..index]. This function assumes 
        that the array is preprocessed and partial sums of 
        array elements are stored in BITree[].
    '''
    #index in BIT is 1 more than array , we dont care about index 0 in BITree    
    index +=1
    s=0
    #Traverse ancestors of BITree[index]
    while (index>0):
        s += BITree[index]
        index -= (index & (-index))#  remove the last set bit, "up and left" in explanation.
    return s

def ADD(BITree,n, index, val):
    '''
    add val to index and update the BITree , n is the length of the array
    '''
    # index in BITree, we dont care about index 0 in BITree
    index += 1 
    # Traverse all ancestors and add 'val'
    while (index <= n):
        BITree[index] += val
        index += (index & (-index))# add the last set bit, " right and up" in explanation.
    return BITree

def constructB(arr,n):
    '''n is the length of arr'''
    # Create and initialize BITree[] as 0
    BITree = [0]*(n+1)  
    for i in range(n):
        ADD(BITree = BITree,n = n, index = i, val = arr[i])
    return BITree
        
# test 
arr =  [2, 1, 1, 3, 2, 3, 4, 5, 6, 7, 8, 9]
BITree = constructB(arr, len(arr))

print("Sum of elements in arr[0..5] is " + str(sum_BIT(BITree, 5))) 
BITree1 = ADD(BITree, len(arr), 5, 6)# add value 6 to index 5
print("Sum of elements in arr[0..5] is " + str(sum_BIT(BITree1, 5))) 

        

Sum of elements in arr[0..5] is 12
Sum of elements in arr[0..5] is 6


## The following is implemented by Kelleher

In find() function shift operation is used. I will explain them here: 
 
 1. left-shift operation (x<<y): shift "y" bits of x to the left and add "y" zero to the right of x; equivalet 
     to: $ x (2^y)$.
 2. right-shift operation (x>>y): shift "y" bits of x to the rigtht and put zeros to the left of x; equivalent to: $ x/(y^2)$.
 
However, I dont know what is ">>=" operation and python gives me an error for that.  
 
 
 source: https://wiki.python.org/moin/BitwiseOperators

In [112]:

class FenwickTree(object):
    """
    A Fenwick Tree to represent cumulative frequency tables over
    integers. Each index from 1 to max_index initially has a
    zero frequency.
    This is an implementation of the Fenwick tree (also known as a Binary
    Indexed Tree) based on "A new data structure for cumulative frequency
    tables", Software Practice and Experience, Vol 24, No 3, pp 327 336 Mar
    1994. This implementation supports any non-negative frequencies, and the
    search procedure always returns the smallest index such that its cumulative
    frequency <= f. This search procedure is a slightly modified version of
    that presented in Tech Report 110, "A new data structure for cumulative
    frequency tables: an improved frequency-to-symbol algorithm." available at
    https://www.cs.auckland.ac.nz/~peter-f/FTPfiles/TechRep110.ps
    """
    def __init__(self, max_index):
        assert max_index > 0
        self.__max_index = max_index
        self.__tree = [0 for j in range(max_index + 1)]## all zero, the index is one more than the arr
        # Compute the binary logarithm of max_index
        u = self.__max_index
        while u != 0:
            self.__log_max_index = u
            u -= (u & -u)
            
    def get_cumulative_frequency(self, index): ## this is equivalent as sumBIT()
        """
        Returns the cumulative frequency of the specified index.
        """
        assert 0 < index <= self.__max_index
        j = index
        s = 0
        while j > 0:
            s += self.__tree[j]
            j -= (j & -j)## remove the last set bit
        return s
    def get_total(self):
        """
        Returns the total cumulative frequency over all indexes.
        """
        return self.get_cumulative_frequency(self.__max_index)

    def increment(self, index, v):## equivalent to my ADD()
        """
        Increments the frequency of the specified index by the specified
        value.
        """
        assert 0 < index <= self.__max_index
        j = index
        while j <= self.__max_index:
            self.__tree[j] += v
            j += (j & -j)  ## add the last set bit 

    def set_value(self, index, v):
        """
        Sets the frequency at the specified index to the specified value.
        """
        f = self.get_frequency(index)
        self.increment(index, v - f)



    def get_frequency(self, index):
        """
        Returns the frequency of the specified index.
        """
        assert 0 < index <= self.__max_index
        j = index
        v = self.__tree[j]
        p = j & (j - 1)# 
        j -= 1
        while p != j:
            v -= self.__tree[j]
            j = j & (j - 1)## amazing, exactly gives you the number of links  of segment with idex =index
        return v

    def find(self, v):
        """
        Returns the smallest index with cumulative sum >= v.
        """
        j = 0
        s = v
        half = self.__log_max_index
        while half > 0:
            # Skip non-existant entries
            while j + half > self.__max_index:
                half >>= 1 ## dont know what is this operation and python gives me an error.
            k = j + half
            if s > self.__tree[k]:
                j = k
                s -= self.__tree[j]
            half >>= 1
        return j + 1



In [94]:
#example 
arr =  [2, 1, 1, 3, 2, 3, 4, 5, 6, 7, 8, 9]
f= FenwickTree(len(arr))
for i in range(len(arr)):
    f.increment(i+1, arr[i])
print(f.find(6))

4


-1

In [47]:
a=4
b=10
x= bin(a)
y = bin(b)
print("x is %s"%x)
print("y is %s"%y)
print("a<<b is %s"%bin(a<<b))
print("b<<a is %s"%bin(b<<a))
print("a>>b is %s"%bin(a >>= 1))

SyntaxError: invalid syntax (<ipython-input-47-0239bf45f3bb>, line 9)

In [69]:
print( x >>= 1)

SyntaxError: invalid syntax (<ipython-input-69-c0081138d21d>, line 1)

In [98]:
n=5 ; m=10
S= bintrees.AVLTree()
S[0]=n
S[m]=-1
# S[1] = 2
# S[3]=-1

print(S)

AVLTree({0: 5, 10: -1})


In [145]:
# My implementation
class TS:
    def __init__(self,sample_size, num_loci, recombination_rate, max_segments =10, from_ts = "None", model ="hudson"):
        self.model = model
        self.n = sample_size
        self.m = num_loci
        self.r = recombination_rate
        self.max_segments = max_segments
        self.segment_stack = []
        N=1 # we only have one population
        pop_index= 0 
        self.segments = [None for j in range(self.max_segments + 1)]
        for j in range(self.max_segments):
            s = Segment(j + 1)
            self.segments[j + 1] = s
            self.segment_stack.append(s)
        self.L = FenwickTree(self.max_segments)# an empty tree -- zero to all 
        self.S = bintrees.AVLTree() # empty avl tree-
        self.P= Population(pop_index)# id =0; as we have only one pop 
        self.P.set_start_size(self.n) # before: self.P[pop_index].set_start_size(population_sizes[pop_index])
        self.edge_buffer = []
        self.from_ts = from_ts # to initialize from ts or not
        if from_ts is None:
            self.tables.populations.add_row()
            for k in range(self.n):
                j = len(self.tables.nodes)
                x = self.alloc_segment(0, self.m, j, pop_index)
                self.L.set_value(x.index, self.m - 1)
                self.P.add(x)# add individual into the population 
                self.tables.nodes.add_row(
                    flags=msprime.NODE_IS_SAMPLE, time=0, population=pop_index)
                j += 1
            # initialize the AVLtree
            self.S[0] = self.n
            self.S[self.m] = -1
            self.t = 0
        else: 
            ts = msprime.load(from_ts)
            if ts.sequence_length != self.m:
                raise ValueError("Sequence length in from_ts must match")
            if ts.num_populations != N:
                raise ValueError("Number of populations in from_ts must match")
            self.initialise_from_ts(ts) # the function  
        self.num_ca_events = 0
        self.num_re_events = 0
        #I think i dont need this as it is for the population changes and related things. 
        self.modifier_events = [(sys.float_info.max, None, None)]# dont know why it is for--- the 
        #sys.... is maximum float available in pyton 
    def initialise_from_ts(self, ts):
        self.tables = ts.dump_tables()
        root_time = np.max(self.tables.nodes.time)
        self.t = root_time
        
        root_segments_head = [None for _ in range(ts.num_nodes)]
        root_segments_tail = [None for _ in range(ts.num_nodes)]
        last_S = -1
        for tree in ts.trees():
            left, right = tree.interval
            S = 0 if tree.num_roots == 1 else tree.num_roots # this tree.num_roots is active lin in each tree??
            if S != last_S:
                self.S[left] = S
                last_S = S
            # If we have 1 root this is a special case and we don't add in
            # any ancestral segments to the state.
           
            if tree.num_roots > 1:
                for root in tree.roots:
                    population = ts.node(root).population
                    if root_segments_head[root] is None:
                        seg = self.alloc_segment(left, right, root, population)
                        root_segments_head[root] = seg
                        root_segments_tail[root] = seg
                    else:
                        tail = root_segments_tail[root]
                        if tail.right == left:
                            tail.right = right
                        else:
                            seg = self.alloc_segment(left, right, root, population, tail)
                            tail.next = seg
                            root_segments_tail[root] = seg
        self.S[self.m] = -1
        # Insert the segment chains into the algorithm state.
        for node in range(ts.num_nodes):
            seg = root_segments_head[node]
            if seg is not None:
                self.L.set_value(seg.index, seg.right - seg.left - 1)
                self.P.add(seg)
                prev = seg
                seg = seg.next
                while seg is not None:
                    self.L.set_value(seg.index, seg.right - prev.right)
                    prev = seg
                    seg = seg.next
    def alloc_segment(self, left, right, node, pop_index, prev=None, next = None):
        """
        Pops a new segment off the stack and sets its properties.
        """
        s = self.segment_stack.pop()# ready segmenets to use : take one of them 
        s.left = left
        s.right = right
        s.node = node
        s.population = pop_index
        s.next = next
        s.prev = prev
        return s
    
    def free_segment(self, u):
        """
        Frees the specified segment making it ready for reuse and
        setting its weight to zero.
        """
        self.L.set_value(u.index, 0)
        self.segment_stack.append(u)# to reuse 
    def store_node(self, population):
        self.flush_edges()
        self.tables.nodes.add_row(time=self.t, population=population)
    def flush_edges(self):## add edges to the table 
        """
        Flushes the edges in the edge buffer to the table, squashing any adjacent edges.
        """
        if len(self.edge_buffer) > 0:
            parent = len(self.tables.nodes) - 1
            self.edge_buffer.sort(key=lambda e: (e.child, e.left))
            left = self.edge_buffer[0].left
            right = self.edge_buffer[0].right
            child = self.edge_buffer[0].child
            assert self.edge_buffer[0].parent == parent
            for e in self.edge_buffer[1:]:
                assert e.parent == parent
                if e.left != right or e.child != child:
                    self.tables.edges.add_row(left, right, parent, child)
                    left = e.left
                    child = e.child
                right = e.right
            self.tables.edges.add_row(left, right, parent, child)
            self.edge_buffer = []
    def store_edge(self, left, right, parent, child):
        """
        Stores the specified edge to the output tree sequence.
        """
        self.edge_buffer.append(
            msprime.Edge(left=left, right=right, parent=parent, child=child))
        
    def finalise(self):
        """
        Finalises the simulation returns an msprime tree sequence object.
        """
        self.flush_edges()
        ts = self.tables.tree_sequence()
        return ts
    def simulate(self, model='hudson'):
        if self.model == 'hudson':
            self.hudson_simulate()
        elif self.model == 'dtwf':
            self.dtwf_simulate()
        else:
            print("Error: bad model specification -", self.model)
            raise ValueError
        return self.finalise()
    
    def hudson_simulate(self):
        """
        Simulates the algorithm until all loci have coalesced.
        """
        infinity = sys.float_info.max
        while self.P.get_num_ancestors() !=0:#sum(pop.get_num_ancestors() for pop in self.P) != 0:
            self.verify()
            rate = self.r * self.L.get_total()
            t_re = infinity
            if rate != 0:
                t_re = random.expovariate(rate)
            # Common ancestor events occur within demes.
            t_ca = infinity
            #for index, pop in enumerate(self.P):
            t = self.P.get_common_ancestor_waiting_time(self.t)#pop.get_common_ancestor_waiting_time(self.t)
            if t < t_ca:
                t_ca = t
#                 ca_population = index
#             t_mig = infinity
#             # Migration events happen at the rates in the matrix.
#             for j in range(len(self.P)):
#                 source_size = self.P[j].get_num_ancestors()
#                 for k in range(len(self.P)):
#                     rate = source_size * self.migration_matrix[j][k]
#                     if rate > 0:
#                         t = random.expovariate(rate)
#                         if t < t_mig:
#                             t_mig = t
#                             mig_source = j
#                             mig_dest = k
            min_time = min(t_re, t_ca)#,t_mig)
            assert min_time != infinity
            if self.t + min_time > self.modifier_events[0][0]:
                t, func, args = self.modifier_events.pop(0)
                self.t = t
                func(*args)
            else:
                self.t += min_time
                if min_time == t_re:
                    # print("RE EVENT")
                    self.hudson_recombination_event()
                else:#elif min_time == t_ca:
                    # print("CA EVENT")
                    self.common_ancestor_event(ca_population)
#                 else:
#                     # print("MIG EVENT")
#                     self.migration_event(mig_source, mig_dest)
        return self.finalise()    
    def hudson_recombination_event(self):
        """
        Implements a recombination event.
        """
        self.num_re_events += 1
        h = random.randint(1, self.L.get_total())
        # Get the segment containing the h'th link
        y = self.segments[self.L.find(h)]
        k = y.right - self.L.get_cumulative_frequency(y.index) + h - 1
        x = y.prev
        if y.left < k:
            # Make new segment
            z = self.alloc_segment(
                k, y.right, y.node, y.population, None, y.next)
            if y.next is not None:
                y.next.prev = z
            y.next = None
            y.right = k
            self.L.increment(y.index, k - z.right)
        else:
            # split the link between x and y.
            x.next = None
            y.prev = None
            z = y
        self.L.set_value(z.index, z.right - z.left - 1)
        #self.P[z.population].add(z)
        self.P.add(z)
    def print_heaps(self, L):## no idea what is this for 
        copy = list(L)
        ordered = [heapq.heappop(copy) for _ in L]
        print("L = ")
        for l, x in ordered:
            print("\t", l, ":", end="")
            u = x
            s = ""
            while u is not None:
                s += "({0}-{1}->{2}({3}))".format(
                    u.left, u.right, u.node, u.index)
                u = u.next
            print(s)


In [ ]:
t= TS().f()